# Pipeline

- Pre-processing
- Sentence Embedding (try different model)
- Dimensionality Reduction (try different)
- Clustering Algorithm (try different)
- Generate Cluster
- First visualisation
- Automatic cluster labeling
- Second visualisation
- Comparison with old label
- Second attempt to generate name with LLM 
- Comparison
IF I HAVE FINISH EVEYRTHING 
- Neuro-symbolic approach
- Possibility to add sentence in dashboard
- Create a good dashboard

# Libraries

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import csv
from sentence_transformers import SentenceTransformer, util
import torch
from umap import UMAP
import plotly
from sklearn.cluster import AgglomerativeClustering
import tensorflow as tf
import hdbscan
import nltk
import re
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score, normalized_mutual_info_score, adjusted_rand_score
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
from tqdm.notebook import trange
from functools import partial
import plotly.express as px


# Data Preprocessing

In [2]:
data_frame = pd.DataFrame( {'thought': [], 'original_label': []})

## Data CVS 
Enlever les doubles labels

In [8]:
# reframing_dataset
neg_thought = []

# read the dataset and take just the negative thought and the label
data = pd.read_csv("/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/reframing_dataset.csv")
data_frame["thought"] = data["thought"]
data_frame["original_label"] = data["thinking_traps_addressed"]
data_frame = data_frame.drop_duplicates()
data_frame

,thought,original_label
0,Someone I trusted stole something valuable of ...,emotional reasoning
2,She doesn't respect me.,overgeneralizing
4,My friend is ignoring his recently-deceased wife.,disqualifying the positive
5,My friend is ignoring his recently-deceased wife.,mind reading
6,he took me for granted!,"labeling,catastrophizing"
...,...,...
592,I'm so exhausted,negative feeling or emotion
594,ill never be able to find a good job,"fortune telling,catastrophizing"
596,I am being disrespected,"labeling,disqualifying the positive,negative f..."
598,maybe she doesn't like me anymore.,mind reading


## Data JSON

In [9]:
# read the dataset and rename the colonne
with open('/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/thinking_traps.jsonl') as f:
    df = pd.read_json(f, lines=True)
    df = df.rename(columns={"completion": "original_label","prompt": "thought"})
# replace  arrow with splace
for i in range(len(df)):

    df["thought"][i] = df["thought"][i].replace("->", "")

# remove the duplicates
df = df.drop_duplicates()
print(len(df))
df = df.drop(df[df['original_label'] == ' Not distorted'].index)
print(len(df))
df

1077
1000


,thought,original_label
0,I'm always late,Overgeneralization
1,I'm late for the meeting. Everyone will look d...,Mind reading
2,I'm late for the meeting. Everyone will look d...,Fortune telling
3,I'm late for the meeting. This shows what a je...,Labeling
4,I'm late for the meeting. I'll make a fool of ...,Labeling
...,...,...
1072,Not having made rent and having to borrow mone...,Labeling
1073,I’m a loser I hate myself and I’m ready to die,Labeling
1074,People around me do not need my presence I'm w...,Emotional reasoning
1075,I realize that I still don’t have anyone who I...,Overgeneralizing


Merge both dataset


In [10]:
corpus_data_xp = data_frame.append(df,ignore_index=True)
corpus_data = corpus_data_xp.drop_duplicates(subset="thought")
corpus_data = corpus_data.reset_index(drop=True)
corpus_data

/var/folders/td/5r8mr8t14cn_c4w5c8ty48lw0000gn/T/ipykernel_4523/2959235978.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  corpus_data_xp = data_frame.append(df,ignore_index=True)


,thought,original_label
0,Someone I trusted stole something valuable of ...,emotional reasoning
1,She doesn't respect me.,overgeneralizing
2,My friend is ignoring his recently-deceased wife.,disqualifying the positive
3,he took me for granted!,"labeling,catastrophizing"
4,We will not be able to complete the work that ...,catastrophizing
...,...,...
917,Not having made rent and having to borrow mone...,Labeling
918,I’m a loser I hate myself and I’m ready to die,Labeling
919,People around me do not need my presence I'm w...,Emotional reasoning
920,I realize that I still don’t have anyone who I...,Overgeneralizing


Explore quickly the both dataset and merge them

In [18]:
explore = (corpus_data.original_label.value_counts()
                   .to_frame()
                   .reset_index()
                   .rename(columns={'index': 'original_label', 'original_label': 'count'}))
explore.to_html('/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/export/exploration_label.html', index=False)

# Clean text (facultatif)

In [420]:
stop_words = set(stopwords.words('english'))

print(stop_words)

{"hasn't", 'hers', 'other', 'himself', 'if', 'for', 'mightn', "doesn't", 'ma', 'a', 'then', 'i', 'about', 'through', "couldn't", 'should', 'myself', 'nor', "should've", 'both', 'isn', 'with', 'in', 'not', 'do', 'by', 'or', 'the', "wasn't", 'up', 'yourself', 'of', 'ain', 'against', 'will', 'few', 'where', "you've", 'off', 'what', 'be', 'it', 'such', 'those', 'further', 'own', 'been', 'hasn', 'ours', 'yours', "didn't", 'below', 'doesn', 'on', 'has', 'were', 're', 'being', 'into', 'you', 'out', 'to', 'most', 'me', 't', "you'd", 'but', 'over', 'mustn', 'each', "aren't", 'we', 'he', "that'll", 'above', 'don', 'wouldn', 'haven', 'was', 'these', 'before', 'while', 'more', 'same', 'until', 'doing', 'am', 'had', "needn't", 'which', "she's", 'couldn', "mightn't", 'our', 'under', 'yourselves', "mustn't", 'is', "isn't", 'ourselves', 'herself', "won't", 'some', 'needn', "weren't", 'whom', 'did', 'does', 'down', 'who', 'd', 'no', 'during', 'my', 'only', 'her', 'having', "shouldn't", 'itself', 'there

In [474]:
def clean_text(text):
    """
    Summary: turns text input string into list of cleaned word tokens
    
    Arguments:
        text: str of text
    
    Returns:
        lemmatized_words: str, lemmatized words from original text after cleaning
    """
    
    # remove numbers
    clean_text = re.sub(r'[0-9]+', '', text)
    
    # remove punctuation
    clean_text = re.sub(r'[^\w\s]', '', clean_text)
    
    # convert everything to lowercase
    clean_text = clean_text.lower()
    
    # tokenize
    wt = WhitespaceTokenizer()
    words = wt.tokenize(clean_text)
    
    # remove stop words
    cleaned_words = []
    for w in words:
        if w not in stop_words:
            cleaned_words.append(w)
            
    # lemmatize words
    wnl = WordNetLemmatizer()
    wnl_lemmatized_tokens = []
    for token in cleaned_words:
        wnl_lemmatized_tokens.append(wnl.lemmatize(token))
    
    lemmatized_words = ' '.join(wnl_lemmatized_tokens)
    
    return lemmatized_words

In [478]:
corpus_data['cleaned_thought'] = corpus_data['thought'].apply(lambda x: clean_text(x))
corpus_data.head()

,thought,original_label,cleaned_thought
0,Someone I trusted stole something valuable of ...,emotional reasoning,someone trusted stole something valuable mine ...
1,She doesn't respect me.,overgeneralizing,doesnt respect
2,My friend is ignoring his recently-deceased wife.,disqualifying the positive,friend ignoring recentlydeceased wife
3,he took me for granted!,"labeling,catastrophizing",took granted
4,We will not be able to complete the work that ...,catastrophizing,able complete work need done


In [476]:
sample_text = ' '.join(list(corpus_data['thought'].values))

clean_sample_text = ' '.join(list(corpus_data['cleaned_thought'].values))

# tokenize
wt = WhitespaceTokenizer()
clean_tokens = wt.tokenize(clean_sample_text)

freq_dist = FreqDist(clean_tokens)
print(freq_dist)
freq_dist.most_common(500)


<FreqDist with 1691 samples and 6907 outcomes>


[('im', 184),
 ('like', 88),
 ('friend', 87),
 ('never', 81),
 ('get', 80),
 ('feel', 70),
 ('job', 68),
 ('work', 66),
 ('dont', 64),
 ('time', 62),
 ('cant', 51),
 ('didnt', 50),
 ('want', 48),
 ('doesnt', 45),
 ('good', 43),
 ('made', 41),
 ('enough', 40),
 ('people', 40),
 ('bad', 39),
 ('going', 39),
 ('something', 38),
 ('got', 38),
 ('much', 36),
 ('must', 36),
 ('thought', 35),
 ('anything', 34),
 ('know', 34),
 ('go', 34),
 ('person', 32),
 ('think', 30),
 ('one', 30),
 ('make', 29),
 ('done', 26),
 ('failure', 25),
 ('ill', 25),
 ('felt', 25),
 ('angry', 24),
 ('better', 24),
 ('love', 23),
 ('hate', 23),
 ('really', 23),
 ('bos', 23),
 ('sister', 23),
 ('thing', 23),
 ('need', 22),
 ('anymore', 22),
 ('could', 22),
 ('husband', 22),
 ('able', 21),
 ('mother', 21),
 ('care', 21),
 ('day', 21),
 ('always', 21),
 ('week', 21),
 ('life', 20),
 ('home', 20),
 ('someone', 19),
 ('everything', 19),
 ('right', 19),
 ('last', 19),
 ('party', 19),
 ('everyone', 18),
 ('talk', 18),
 ('

In [479]:
word_counts = freq_dist.items()

# convert the tuple pairs into a dictionary
word_freq_map = {word: count for word, count in word_counts}

In [481]:
import pandas as pd

# convert the dictionary to a Pandas DataFrame
df = pd.DataFrame.from_dict(word_freq_map, orient='index', columns=['frequency'])

# reset the index to make the word column
df = df.reset_index()

# sort the DataFrame by frequency in descending order
df = df.sort_values('frequency', ascending=False)

# rename the index column to 'word'
df.columns = ['word', 'frequency']
# print the DataFrame
print(df)


          word  frequency
35          im        184
88        like         88
12      friend         87
48       never         81
73         get         80
...        ...        ...
1068    canned          1
1069       air          1
1070      loud          1
1071     noise          1
1690  maintain          1

[1691 rows x 2 columns]


In [482]:
import plotly.express as px
df["frequency"]
fig = px.line(df, x="word", y="frequency", title='Life expectancy in Canada')
fig.show()

In [483]:
corpus_data

,thought,original_label,cleaned_thought
0,Someone I trusted stole something valuable of ...,emotional reasoning,someone trusted stole something valuable mine ...
1,She doesn't respect me.,overgeneralizing,doesnt respect
2,My friend is ignoring his recently-deceased wife.,disqualifying the positive,friend ignoring recentlydeceased wife
3,he took me for granted!,"labeling,catastrophizing",took granted
4,We will not be able to complete the work that ...,catastrophizing,able complete work need done
...,...,...,...
917,Not having made rent and having to borrow mone...,Labeling,made rent borrow money already debt mother fai...
918,I’m a loser I hate myself and I’m ready to die,Labeling,im loser hate im ready die
919,People around me do not need my presence I'm w...,Emotional reasoning,people around need presence im worthless
920,I realize that I still don’t have anyone who I...,Overgeneralizing,realize still dont anyone could spend time wit...


# Analysis

## Sentence embeddings
https://www.sbert.net/index.html

In [19]:
"""
Sentences are mapped to sentence embeddings.

"""

# pick the corpus of negative thought
corpus = list(corpus_data["thought"])

# specify the model
model = 'all-mpnet-base-v2'
model_2 = 'all-MiniLM-L12-v2'
model_3 = 'All-Distilroberta-v1'

# Load the model
embedder = SentenceTransformer(model)
embedder_2 = SentenceTransformer(model_2)
embedder_3 = SentenceTransformer(model_3)

# encode
corpus_embeddings = embedder.encode(corpus)
corpus_embeddings_2 = embedder_2.encode(corpus)
corpus_embeddings_3 = embedder_3.encode(corpus)

# merge 3 embeding
embeddings = [corpus_embeddings, corpus_embeddings_2, corpus_embeddings_3]

print(corpus_embeddings.shape)
print(corpus_embeddings_2.shape)
print(corpus_embeddings_3.shape)

(922, 768)
(922, 384)
(922, 768)


## Clustering
Without using dimension reduction test


### function
differents function take from : https://github.com/dborrelli/chat-intents/blob/main/notebooks/04-umap_hdbscan.ipynb for the HDBSCAN part


In [20]:
def plot_kmeans(embeddings, k_range):
    '''
    Plot SSE and silhouette score for kmeans clustering for a range of k values
    Arguments:
        embeddings: array, sentence embeddings
        k_range: range, values of k to evaluate for kmeans clustering  
    '''
    sse = []
    silhouette_avg_n_clusters = []

    for k in k_range:
        kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
        kmeans.fit(embeddings)
        sse.append(kmeans.inertia_)

        silhouette_avg = silhouette_score(embeddings, kmeans.predict(embeddings))
        silhouette_avg_n_clusters.append(silhouette_avg)


    # plot sse
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    axes[0].plot(k_range, sse)
    axes[0].set(xlabel = 'k clusters', ylabel = 'SSE', title = 'Elbow plot')
    axes[0].grid()
    
    # plot avg silhouette score
    axes[1].plot(k_range, silhouette_avg_n_clusters)
    axes[1].set(xlabel = 'k clusters', ylabel = 'Silhouette score', title = 'Silhouette score')
    axes[1].grid()

    plt.show()

In [21]:
def search_cluster_size(embeddings, size_range):
    '''
    Scan HDBSCAN min_cluster_size values and return results
    Arguments:
        embeddings: embeddings to use
        size_range: range of min_cluster_size hyperparameter values to scan
    Returns:
        result_df: dataframe of min_cluster_size, total number of clusters,
                   and percent of data labeled as noise
    '''
    
    results = []
    
    for i in size_range:
        min_cluster_size = i
        
        clusters_hdbscan = (hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                                           metric='euclidean', 
                                           cluster_selection_method='eom') 
                                           .fit(embeddings))
    
        labels = clusters_hdbscan.labels_
        label_count = len(np.unique(labels))
        
        total_num = len(clusters_hdbscan.labels_)
        cost = (np.count_nonzero(clusters_hdbscan.probabilities_ < 0.05)/total_num) 
                
        results.append([min_cluster_size, label_count, cost])
    
    result_df = pd.DataFrame(results, columns=['min_cluster_size', 
                                               'label_count', 'noise'])
    
    return result_df

In [22]:
def random_search(embeddings, space, num_evals):
    '''
    Random search of HDBSCAN hyperparameter spcae
    Arguments:
        embeddings: embeddings to use
        space: dict, hyperparameter space to search with keys of
               'min_cluster_size' and 'min_samples' and values as ranges
        num_evals: int, number of trials to run
    Returns:
        result_df: dataframe of run_id, min_cluster_size, min_samples, 
                   total number of clusters, and percent of data labeled as noise
    
    '''
    
    results = []
    
    for i in range(num_evals):
        min_cluster_size = random.choice(space['min_cluster_size'])
        min_samples = random.choice(space['min_samples'])
        
        clusters_hdbscan = (hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                                           min_samples = min_samples,
                                           metric='euclidean', 
                                           cluster_selection_method='eom') 
                                           .fit(embeddings))
    
        labels = clusters_hdbscan.labels_
        label_count = len(np.unique(labels))
        
        total_num = len(clusters_hdbscan.labels_)
        cost = (np.count_nonzero(clusters_hdbscan.probabilities_ < 0.05)/total_num) 
                
        results.append([i, min_cluster_size, min_samples, label_count, cost])
    
    result_df = pd.DataFrame(results, columns=['run_id', 'min_cluster_size', 'min_samples', 
                                               'label_count', 'cost'])
    
    return result_df.sort_values(by='cost')

In [26]:
def combine_results(df_ground, cluster_dict):
    """
    Returns dataframe of all documents and each model's assigned cluster

    Arguments:
        df_ground: dataframe of original documents with associated ground truth
                   labels
        cluster_dict: dict, keys as column name for specific model and value as
                      best clusters HDBSCAN object

    Returns:
        df_combined: dataframe of all documents with labels from
                     best clusters for each model

    """

    df_combined = df_ground.copy()
    
    for key, value in cluster_dict.items():
        df_combined[key] = value.labels_
    
    return df_combined

In [27]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given clustering

    Arguments:
        clusters: HDBSCAN clustering object
        prob_threshold: float, probability threshold to use for deciding
                        what cluster labels are considered low confidence

    Returns:
        label_count: int, number of unique cluster labels, including noise
        cost: float, fraction of data points whose cluster assignment has
              a probability below cutoff threshold
    """
    
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)
    
    return label_count, cost

In [28]:
def random_search(embeddings, space, num_evals):
    """
    Randomly search parameter space of clustering pipeline

    Arguments:
        embeddings: embeddings to use
        space: dict, contains keys for 'n_neighbors', 'n_components',
               and 'min_cluster_size' and values with
               corresponding lists or ranges of parameters to search
        num_evals: int, number of random parameter combinations to try

    Returns:
        df_result: pandas dataframe containing info on each evaluation
                   performed, including run_id, parameters used, label
                   count, and cost
    """
    
    results = []
    
    for i in range(num_evals):
        n_neighbors = random.choice(space['n_neighbors'])
        n_components = random.choice(space['n_components'])
        min_cluster_size = random.choice(space['min_cluster_size'])
        random_state = space['random_state']
        
        clusters = generate_clusters(embeddings, 
                                     n_neighbors=n_neighbors, 
                                     n_components=n_components, 
                                     min_cluster_size=min_cluster_size, 
                                     random_state=random_state)
    
        label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
                
        results.append([i, n_neighbors, n_components, min_cluster_size, label_count, cost])
    
    result_df = pd.DataFrame(results, columns=['run_id', 'n_neighbors', 'n_components', 
                                               'min_cluster_size', 'label_count', 'cost'])
    
    return result_df.sort_values(by='cost')

In [29]:
def objective(params, embeddings, label_lower, label_upper):
    """
    Objective function for hyperopt to minimize

    Arguments:
        params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'random_state' and
               their values to use for evaluation
        embeddings: embeddings to use
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters

    Returns:
        loss: cost function result incorporating penalties for falling
              outside desired range for number of clusters
        label_count: int, number of unique cluster labels, including noise
        status: string, hypoeropt status

        """
    
    clusters = generate_clusters(embeddings, 
                                 n_neighbors = params['n_neighbors'], 
                                 n_components = params['n_components'], 
                                 min_cluster_size = params['min_cluster_size'],
                                 random_state = params['random_state'])
    
    label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
    
    #15% penalty on the cost function if outside the desired range of groups
    if (label_count < label_lower) | (label_count > label_upper):
        penalty = 0.15 
    else:
        penalty = 0
    
    loss = cost + penalty
    
    return {'loss': loss, 'label_count': label_count, 'status': STATUS_OK}


In [30]:
def bayesian_search(embeddings, space, label_lower, label_upper, max_evals=100):
    """
    Perform bayesian search on hyperparameter space using hyperopt

    Arguments:
        embeddings: embeddings to use
        space: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', and 'random_state' and
               values that use built-in hyperopt functions to define
               search spaces for each
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters
        max_evals: int, maximum number of parameter combinations to try

    Saves the following to instance variables:
        best_params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'min_samples', and 'random_state' and
               values associated with lowest cost scenario tested
        best_clusters: HDBSCAN object associated with lowest cost scenario
                       tested
        trials: hyperopt trials object for search

        """
    
    trials = Trials()
    fmin_objective = partial(objective, 
                             embeddings=embeddings, 
                             label_lower=label_lower,
                             label_upper=label_upper)
    
    best = fmin(fmin_objective, 
                space = space, 
                algo=tpe.suggest,
                max_evals=max_evals, 
                trials=trials)

    best_params = space_eval(space, best)
    print ('best:')
    print (best_params)
    print (f"label count: {trials.best_trial['result']['label_count']}")
    
    best_clusters = generate_clusters(embeddings, 
                                      n_neighbors = best_params['n_neighbors'], 
                                      n_components = best_params['n_components'], 
                                      min_cluster_size = best_params['min_cluster_size'],
                                      random_state = best_params['random_state'])
    
    return best_params, best_clusters, trials


In [31]:
def summarize_results(results_dict, results_df):
    """
    Returns a table summarizing each model's performance compared to ground
    truth labels and the model's hyperparametes

    Arguments:
        results_dict: dict, key is the model name and value is a list of: 
                      model column name in combine_results output, best_params and best_clusters 
                      for each model (e.g. ['label_use', best_params_use, trials_use])
        results_df: dataframe output of combine_results function; dataframe of all documents 
                    with labels from best clusters for each model

    Returns:
        df_final: dataframe with each row including a model name, calculated ARI and NMI,
                  loss, label count, and hyperparameters of best model

    """
    
    summary = []

    for key, value in results_dict.items():
        ground_label = results_df['original_label'].values
        predicted_label = results_df[value[0]].values
        
        ari = np.round(adjusted_rand_score(ground_label, predicted_label), 3)
        nmi = np.round(normalized_mutual_info_score(ground_label, predicted_label), 3)
        loss = value[2].best_trial['result']['loss']
        label_count = value[2].best_trial['result']['label_count']
        n_neighbors = value[1]['n_neighbors']
        n_components = value[1]['n_components']
        min_cluster_size = value[1]['min_cluster_size']
        random_state = value[1]['random_state']
        
        summary.append([key, ari, nmi, loss, label_count, n_neighbors, n_components, 
                        min_cluster_size, random_state])

    df_final = pd.DataFrame(summary, columns=['Model', 'ARI', 'NMI', 'loss', 
                                              'label_count', 'n_neighbors',
                                              'n_components', 'min_cluster_size',
                                              'random_state'])
    
    return df_final.sort_values(by='NMI', ascending=False)


In [32]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components, 
                      min_cluster_size,
                      min_samples = None,
                      random_state = None):
    """
    Returns HDBSCAN objects after first performing dimensionality reduction using UMAP
    
    Arguments:
        message_embeddings: embeddings to use
        n_neighbors: int, UMAP hyperparameter n_neighbors
        n_components: int, UMAP hyperparameter n_components
        min_cluster_size: int, HDBSCAN hyperparameter min_cluster_size
        min_samples: int, HDBSCAN hyperparameter min_samples
        random_state: int, random seed
        
    Returns:
        clusters: HDBSCAN object of clusters
    """
    
    umap_embeddings = (umap.UMAP(n_neighbors = n_neighbors, 
                                n_components = n_components, 
                                metric = 'cosine', 
                                random_state=random_state)
                            .fit_transform(message_embeddings))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size, 
                               min_samples = min_samples,
                               metric='euclidean', 
                               gen_min_span_tree=True,
                               cluster_selection_method='eom').fit(umap_embeddings)
    
    return clusters

## K-mean clustering




### clustering

In [33]:
# Parameters
cluster_dict_k = {}
n_clusters = 18 # number of cluster
dimension_reduction = True # reduce the dimension before clustering
dim = 2 # number of dimension to reduce embedding to.
#corpus_embeddings
#corpus_embeddings_2
#corpus_embeddings_3
corpus_embeddings_reduce = corpus_embeddings

In [35]:
# Dimension reduction
if dimension_reduction:
    # Reduce the embedding with UMAP
    model = UMAP(n_components=dim)
    corpus_embeddings_reduce = model.fit_transform(corpus_embeddings)

print(corpus_embeddings_reduce.shape)

# Cluster with KMeans
kmeans = KMeans(n_clusters=n_clusters)
cluster = kmeans.fit(corpus_embeddings_reduce)
cluster_dict_k["k-mean"] =  cluster

# Get cluster labels
cluster_labels = cluster.labels_

# Reduce the dimension with UMAP for visualization
model2 = UMAP(n_components=2)
embeddings_2d = model2.fit_transform(corpus_embeddings_reduce)
x, y = embeddings_2d.T  

(922, 2)


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


### plot k-mean

In [38]:
# Plot
# Create a DataFrame for Plotly Express
plot_data = {
    'x': x,
    'y': y,
    'cluster_labels': cluster_labels,
    'hover_text': corpus  # text you want to display
}

df = pd.DataFrame(plot_data)

# Convert cluster_labels to string for categorical representation
df['cluster_labels'] = df['cluster_labels'].astype(str)

# plot using Plotly Express
fig = px.scatter(
    df,
    x='x',
    y='y',
    color='cluster_labels',
    hover_data={'hover_text': True},  # Enable hover data for 'hover_text'
    title='Cluster of negative thought using K-mean'
)

fig.show()
# Export the plot to HTML
fig.write_html(f"/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/export/k_mean.html")
results_k_mean = combine_results(corpus_data[['thought', 'original_label']], cluster_dict_k)



### result k-mean

In [45]:
results_k_mean[results_k_mean['k-mean']== 0]


,thought,original_label,k-mean
6,Am I not explaining the material well?,"all-or-nothing thinking,personalizing",0
46,I should have been perfect,should statements,0
62,I didn't do as good a job as I thought,disqualifying the positive,0
69,I forgot something important. Again. My boss i...,"catastrophizing,fortune telling",0
76,Why did I overact like this,catastrophizing,0
78,I messed up big time.,catastrophizing,0
80,they should have reminded me before I forgot,should statements,0
85,I could have done so much better,"personalizing,all-or-nothing thinking",0
108,I'm mad at myself for being careless,"emotional reasoning,labeling,negative feeling ...",0
111,I screwed up again at work,"negative feeling or emotion,all-or-nothing thi...",0


#### Generate name (en cours)

In [162]:
# Here I take each sentence from one cluster and I am generating a label for theses sentences
n_clusters
for clust in range(0,n_clusters):
    len_clus = len(df[df['cluster_labels'] == str(clust)])
    df[df['cluster_labels'] == str(clust)]
    print(f"la longueur du cluster {clust} est {len_clus}")
    
sentence_class = df[df['cluster_labels'] == '5'].hover_text
list(sentence_class)


la longueur du cluster 0 est 64
la longueur du cluster 1 est 47
la longueur du cluster 2 est 37
la longueur du cluster 3 est 36
la longueur du cluster 4 est 53
la longueur du cluster 5 est 49
la longueur du cluster 6 est 47
la longueur du cluster 7 est 73
la longueur du cluster 8 est 52
la longueur du cluster 9 est 63
la longueur du cluster 10 est 45
la longueur du cluster 11 est 47
la longueur du cluster 12 est 54
la longueur du cluster 13 est 44
la longueur du cluster 14 est 57
la longueur du cluster 15 est 71
la longueur du cluster 16 est 58
la longueur du cluster 17 est 25


["I'm not smart enough to succeed at mturk",
 "I'm a failure",
 "I'm wasting all my time",
 'I am not good enough for anyone.',
 "I'm incompetent",
 "I'll never be able to do anything",
 'I feel like a failure',
 "I'm not succeeding",
 'I keep failing at this',
 'I am not good at what I do.',
 'I have lost all hope',
 "I can't sing.",
 'I am not good enough.',
 'Other people are probably doing it better than me.',
 'This work is a failure',
 "I'm really bad at everything I do.",
 'Am I doing enough',
 "I can't do anything right. I am a disappointment.",
 "I'm worried about my future",
 'I bet all my friends will be amazing at bowling apart from me',
 "I've been practicing but I'm not making any progress",
 "I don't know why I keep trying. I should just give up. Surrender. Sleep forever.",
 "I'm not going to live up to my expectations for my future",
 "I can't get through this",
 'I never get what I want',
 'I am unable to do this simply damned thing.',
 'I am a failure',
 "i'm not good

## AgglomerativeClustering



In [61]:
# Parameters
dimension_reduction = True # reduce the dimension before clustering
dim = 3 # number of dimension to reduce embedding to.

In [63]:
# Dimension reduction
corpus_embeddings_reduce=corpus_embeddings
if dimension_reduction:
    # Reduce the embedding with UMAP
    model = UMAP(n_components=dim)
    corpus_embeddings_reduce = model.fit_transform(corpus_embeddings)
    print(corpus_embeddings_reduce.shape)
print(corpus_embeddings_reduce.shape)

corpus_embeddings_reduce = corpus_embeddings_reduce /  np.linalg.norm(corpus_embeddings_reduce, axis=1, keepdims=True)

# Cluster with Agglomerative
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5)
cluster = clustering_model.fit(corpus_embeddings_reduce)
cluster_dict_k["aglo"] =  cluster
# Get cluster labels
cluster_labels = cluster.labels_

# Reduce the dimension with UMAP for visualization
model2 = UMAP(n_components=2)
embeddings_2d = model2.fit_transform(corpus_embeddings_reduce)
x, y = embeddings_2d.T  # Use the already fitted embeddings_2d for x, y coordinates


(922, 3)
(922, 3)


In [67]:
# Plot
# Create a DataFrame for Plotly Express
plot_data = {
    'x': x,
    'y': y,
    'cluster_labels': cluster_labels,
    'hover_text': corpus  # text you want to display
}

df = pd.DataFrame(plot_data)


# Convert cluster_labels to string for categorical representation
df['cluster_labels'] = df['cluster_labels'].astype(str)

# plot using Plotly Express
fig = px.scatter(
    df,
    x='x',
    y='y',
    color='cluster_labels',
    hover_data={'hover_text': True},  # Enable hover data for 'hover_text'
    title='AgglomerativeClustering of negative though'
)
fig.show()
# Export the plot to HTML
fig.write_html(f"/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/export/aglo.html")
results_k_mean = combine_results(corpus_data[['thought', 'original_label']], cluster_dict_k)


## HDBSCAN


In [68]:
cluster_dict = {}

### Random Estimation & Manuel

I try a random search of the parameter to find a good compromise between noise and number of cluster

In [70]:
import umap
space = {
        "n_neighbors": range(12,16),
        "n_components": range(3,7),
        "min_cluster_size": range(2,16),
        "random_state": 42
    }
random_dict = {}
a=0
for embedding in embeddings:
    a=a+1
    embedding_name = f"embedding_{a}"
    random_dict[embedding_name] = random_search(embedding,space, 100)

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/sylvainestebe/Code/nlp_proje

In [71]:
random_dict["embedding_3"].sort_values(by='cost').head(20)


,run_id,n_neighbors,n_components,min_cluster_size,label_count,cost
76,76,15,4,13,3,0.072668
97,97,15,3,10,5,0.104121
65,65,14,3,13,4,0.105206
77,77,12,4,2,120,0.219089
75,75,12,6,2,118,0.235358
19,19,12,5,2,122,0.236443
78,78,15,3,2,132,0.240781
33,33,14,4,2,122,0.242950
74,74,14,4,2,122,0.242950
39,39,14,4,2,122,0.242950


In [732]:
random_dict["embedding_2"].sort_values(by='cost').head(50)


,run_id,n_neighbors,n_components,min_cluster_size,label_count,cost
5,5,14,4,14,4,0.215835
60,60,14,4,14,4,0.215835
86,86,15,4,2,131,0.218004
63,63,14,3,2,135,0.223427
94,94,12,4,2,141,0.234273
77,77,14,6,2,125,0.235358
39,39,14,4,2,128,0.235358
41,41,13,3,2,138,0.245119
19,19,14,5,2,127,0.249458
66,66,14,3,3,73,0.253796


In [72]:
random_dict["embedding_1"].sort_values(by='cost').head(50)


,run_id,n_neighbors,n_components,min_cluster_size,label_count,cost
49,49,15,3,9,2,0.000000
23,23,12,5,11,2,0.000000
8,8,13,6,8,2,0.000000
97,97,13,6,13,2,0.000000
89,89,12,3,7,2,0.000000
5,5,13,6,12,2,0.000000
64,64,15,3,9,2,0.000000
58,58,13,6,14,2,0.000000
74,74,12,5,9,2,0.000000
25,25,12,3,7,2,0.000000


In [74]:
# model1 corpus_embeddings all-mpnet-base-v2
clusters_manual = generate_clusters(corpus_embeddings,n_neighbors = 14, 
                                              n_components = 3, 
                                              min_cluster_size = 11, 
                                              min_samples = None,
                                              random_state=42)
labels_manual, cost_manual = score_clusters(clusters_manual)
print(labels_manual)
print(cost_manual)
cluster_labels  = clusters_manual.labels_
cluster_dict["label_manual1"] =  clusters_manual

umap_data = umap.UMAP(n_neighbors=15, 
                          n_components=2, 
                          min_dist = 0.1,  
                          metric='cosine',
                          random_state=42).fit_transform(corpus_embeddings)
x, y = umap_data.T
                    # Plot
            # Create a DataFrame for Plotly Express
plot_data = {
                'x': x,
                'y': y,
                'cluster_labels': cluster_labels,
                'hover_text': corpus  # text you want to display
            }

df = pd.DataFrame(plot_data)



            # Convert cluster_labels to string for categorical representation
df['cluster_labels'] = df['cluster_labels'].astype(str)

            # plot using Plotly Express
fig = px.scatter(
                df,
                x='x',
                y='y',
                color='cluster_labels',
                hover_data={'hover_text': True},  # Enable hover data for 'hover_text'
                title='HDBSCAN all-mpnet-base-v2'
            )

            # Export the plot to HTML
fig.write_html(f"/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/export/cluster_manual_all-mpnet-base-v2.html")
fig.show()


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



8
0.1876355748373102


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [75]:
#model_2 = 'all-MiniLM-L12-v2'
# 15	5	9	18	0.332972

# corpus_embeddings all-mpnet-base-v2
clusters_manual = generate_clusters(corpus_embeddings_2,n_neighbors = 15, 
                                              n_components = 5, 
                                              min_cluster_size = 9, 
                                              min_samples = None,
                                              random_state=42)
labels_manual, cost_manual = score_clusters(clusters_manual)
print(labels_manual)
print(cost_manual)
cluster_labels  = clusters_manual.labels_
cluster_dict["label_manual2"] =  clusters_manual

umap_data = umap.UMAP(n_neighbors=15, 
                          n_components=2, 
                          min_dist = 0.1,  
                          metric='cosine',
                          random_state=42).fit_transform(corpus_embeddings_2)
x, y = umap_data.T
                    # Plot
            # Create a DataFrame for Plotly Express
plot_data = {
                'x': x,
                'y': y,
                'cluster_labels': cluster_labels,
                'hover_text': corpus  # text you want to display
            }

df = pd.DataFrame(plot_data)



            # Convert cluster_labels to string for categorical representation
df['cluster_labels'] = df['cluster_labels'].astype(str)

            # plot using Plotly Express
fig = px.scatter(
                df,
                x='x',
                y='y',
                color='cluster_labels',
                hover_data={'hover_text': True},  # Enable hover data for 'hover_text'
                title='HDBSCAN all-MiniLM-L12-v2'
            )

            # Export the plot to HTML
fig.write_html(f"//Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/export/cluster_manual2_all-MiniLM-L12-v2.html")
fig.show()



/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



18
0.3329718004338395


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [76]:
#model_3 = 'All-Distilroberta-v1
# 12	5	10 18	0.287419

# corpus_embeddings all-mpnet-base-v2
clusters_manual = generate_clusters(corpus_embeddings_3,n_neighbors = 12, 
                                              n_components = 5, 
                                              min_cluster_size = 10, 
                                              min_samples = None,
                                              random_state=42)
labels_manual, cost_manual = score_clusters(clusters_manual)
print(labels_manual)
print(cost_manual)
cluster_labels  = clusters_manual.labels_
cluster_dict["label_manual3"] =  clusters_manual

umap_data = umap.UMAP(n_neighbors=15, 
                          n_components=2, 
                          min_dist = 0.1,  
                          metric='cosine',
                          random_state=42).fit_transform(corpus_embeddings_3)
x, y = umap_data.T
                    # Plot
            # Create a DataFrame for Plotly Express
plot_data = {
                'x': x,
                'y': y,
                'cluster_labels': cluster_labels,
                'hover_text': corpus  # text you want to display
            }

df = pd.DataFrame(plot_data)



            # Convert cluster_labels to string for categorical representation
df['cluster_labels'] = df['cluster_labels'].astype(str)

            # plot using Plotly Express
fig = px.scatter(
                df,
                x='x',
                y='y',
                color='cluster_labels',
                hover_data={'hover_text': True},  # Enable hover data for 'hover_text'
                title='Your Title'
            )

            # Export the plot to HTML
fig.write_html(f"/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/export/cluster_manual_All-Distilroberta-v1.html")
fig.show()



/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



18
0.2874186550976139


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



### Bayesian Estimation

In [78]:
best_params_dict = {}
best_clusters_dict = {}
trials_dict = {}

hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(2,16)),
    "n_components": hp.choice('n_components', range(2,10)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,11)),
    "random_state": 42
}

label_lower = 5
label_upper = 60
max_evals = 100

In [79]:
def cluster_to_data(embeddings,plot):
    a=0
    for embedding in embeddings:
        a=a+1
        embedding_name = f"embedding_{a}"
        best_params_use, best_clusters_use, trials_use = bayesian_search(embedding, 
                                                                    space=hspace, 
                                                                    label_lower=label_lower, 
                                                                    label_upper=label_upper, 
                                                      max_evals=max_evals)
        best_params_dict[embedding_name] = best_params_use
        best_clusters_dict[embedding_name] = best_clusters_use
        trials_dict[embedding_name] = trials_use
        cluster_dict[f"label_use{a}"] =    best_clusters_dict[embedding_name]
        cluster_labels = best_clusters_use.labels_
        
        if plot:

            umap_data = umap.UMAP(n_neighbors=15, 
                          n_components=2, 
                          min_dist = 0.1,  
                          metric='cosine',
                          random_state=42).fit_transform(embedding)
            x, y = umap_data.T
                    # Plot
            # Create a DataFrame for Plotly Express
            plot_data = {
                'x': x,
                'y': y,
                'cluster_labels': cluster_labels,
                'hover_text': corpus  # text you want to display
            }

            df = pd.DataFrame(plot_data)



            # Convert cluster_labels to string for categorical representation
            df['cluster_labels'] = df['cluster_labels'].astype(str)

            # plot using Plotly Express
            fig = px.scatter(
                df,
                x='x',
                y='y',
                color='cluster_labels',
                hover_data={'hover_text': True},  # Enable hover data for 'hover_text'
                title=f'Bayésian Estimation {a}'
            )

            # Export the plot to HTML
            fig.write_html(f"/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/export/plot_bayésian{a}.html")
            fig.show()
        results_df = combine_results(corpus_data[['thought', 'original_label']], cluster_dict)
    return results_df,best_params_dict,best_clusters_dict,trials_dict


In [80]:
results_df,best_params_dict,best_clusters_dict,trials_dict = cluster_to_data(embeddings,True)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.




  1%|          | 1/100 [00:01<02:24,  1.46s/trial, best loss: 0.3524945770065076]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.




  2%|▏         | 2/100 [00:02<02:21,  1.44s/trial, best loss: 0.3481561822125813]

...

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.




 99%|█████████▉| 99/100 [02:08<00:01,  1.39s/trial, best loss: 0.03796095444685466]

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.




100%|██████████| 100/100 [02:09<00:00,  1.30s/trial, best loss: 0.03796095444685466]
best:
{'min_cluster_size': 10, 'n_components': 8, 'n_neighbors': 3, 'random_state': 42}
label count: 6


/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

/Users/sylvainestebe/Code/nlp_project/.venv/lib/python3.9/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [81]:
model_dict = {'all-mpnet-base-v2': ['label_use1', best_params_dict["embedding_1"], trials_dict["embedding_1"]],
              'all-MiniLM-L6-v2': ['label_use2', best_params_dict["embedding_2"], trials_dict["embedding_2"]],
              'all-distilroberta-v1': ['label_use3', best_params_dict["embedding_3"], trials_dict["embedding_3"]]               }


In [83]:
results_df

,thought,original_label,label_manual1,label_manual2,label_manual3,label_use1,label_use2,label_use3
0,Someone I trusted stole something valuable of ...,emotional reasoning,2,-1,1,3,4,4
1,She doesn't respect me.,overgeneralizing,4,0,6,3,4,4
2,My friend is ignoring his recently-deceased wife.,disqualifying the positive,3,0,6,3,4,4
3,he took me for granted!,"labeling,catastrophizing",3,0,6,3,4,4
4,We will not be able to complete the work that ...,catastrophizing,2,12,10,3,4,4
...,...,...,...,...,...,...,...,...
917,Not having made rent and having to borrow mone...,Labeling,2,15,-1,3,4,4
918,I’m a loser I hate myself and I’m ready to die,Labeling,2,-1,-1,3,4,4
919,People around me do not need my presence I'm w...,Emotional reasoning,2,15,-1,3,4,4
920,I realize that I still don’t have anyone who I...,Overgeneralizing,2,-1,-1,3,4,4


In [85]:
summarize_results(model_dict, results_df)


,Model,ARI,NMI,loss,label_count,n_neighbors,n_components,min_cluster_size,random_state
1,all-MiniLM-L6-v2,-0.000,0.064,0.035792,6,4,2,8,42
2,all-distilroberta-v1,-0.002,0.058,0.037961,6,3,8,10,42
0,all-mpnet-base-v2,0.000,0.036,0.001085,5,10,6,6,42


In [86]:
results_df[results_df['original_label']== "emotional reasoning"]


,thought,original_label,label_manual1,label_manual2,label_manual3,label_use1,label_use2,label_use3
0,Someone I trusted stole something valuable of ...,emotional reasoning,2,-1,1,3,4,4
9,Suicidal thoughts,emotional reasoning,2,16,15,3,4,4
15,I am scared of my future life and being in sor...,emotional reasoning,-1,2,-1,3,4,4
29,I am being annoying.,emotional reasoning,2,-1,-1,3,4,4
116,I felt that my personal self was under attack ...,emotional reasoning,-1,13,1,3,4,4
154,What am I doing to deserve this?,emotional reasoning,-1,-1,-1,3,3,4
164,I can't be a mother,emotional reasoning,5,3,5,3,4,4
176,I feel like I will never be loved in a romanti...,emotional reasoning,2,6,4,3,4,1
182,My parents are judging me,emotional reasoning,2,-1,4,3,4,4
185,I feel guilty,emotional reasoning,2,4,5,3,4,-1


# Generate label

In [96]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [99]:
data_clustered = results_df[['thought', 'label_use1']]
data_clustered_k_mean = results_k_mean[['thought', 'k-mean']]
data_clustered_k_mean.sample(10)

,thought,k-mean
773,At work I was frustrated after a meeting becau...,15
234,I don't matter.,2
610,I saw a profile picture of a close friend on a...,8
163,There will never be enough time to get everyth...,16
262,I hate Republicans,4
95,I am selfish,2
475,"If I talk, I will mess up and not say what I m...",9
436,I'm a loser,7
255,I felt the criticism was unfair because they d...,15
627,They did not convert me as permanent at work I...,11


In [102]:
example_category = data_clustered_k_mean[data_clustered_k_mean['k-mean']==0].reset_index(drop=True)
example_category 

,thought,k-mean
0,Am I not explaining the material well?,0
1,I should have been perfect,0
2,I didn't do as good a job as I thought,0
3,I forgot something important. Again. My boss i...,0
4,Why did I overact like this,0
5,I messed up big time.,0
6,they should have reminded me before I forgot,0
7,I could have done so much better,0
8,I'm mad at myself for being careless,0
9,I screwed up again at work,0


In [112]:
import spacy
from spacy import displacy

example_doc = nlp(list(example_category['thought'])[12])

print(f'{example_doc}\n')


I must not have been doing a good job



In [106]:
def get_group(df, category_col, category):
    """
    Returns documents of a single category
    
    Arguments:
        df: pandas dataframe of documents
        category_col: str, column name corresponding to categories or clusters
        category: int, cluster number to return
    Returns:
        single_category: pandas dataframe with documents from a single category
    """
    
    single_category = df[df[category_col]==category].reset_index(drop=True)

    return single_category

In [107]:
def most_common(lst, n_words):
    """
    Get most common words in a list of words
    
    Arguments:
        lst: list, each element is a word
        n_words: number of top common words to return
    
    Returns:
        counter.most_common(n_words): counter object of n most common words
    """
    counter=collections.Counter(lst)
    return counter.most_common(n_words)


In [108]:
def extract_labels(category_docs, print_word_counts=False):
    """
    Extract labels from documents in the same cluster by concatenating
    most common verbs, ojects, and nouns

    Argument:
        category_docs: list of documents, all from the same category or
                       clustering
        print_word_counts: bool, True will print word counts of each type in this category

    Returns:
        label: str, group label derived from concatentating most common
               verb, object, and two most common nouns

    """

    verbs = []
    dobjs = []
    nouns = []
    adjs = []
    
    verb = ''
    dobj = ''
    noun1 = ''
    noun2 = ''

    # for each document, append verbs, dobs, nouns, and adjectives to 
    # running lists for whole cluster
    for i in range(len(category_docs)):
        doc = nlp(category_docs[i])
        for token in doc:
            if token.is_stop==False:
                if token.dep_ == 'ROOT':
                    verbs.append(token.text.lower())

                elif token.dep_=='dobj':
                    dobjs.append(token.lemma_.lower())

                elif token.pos_=='NOUN':
                    nouns.append(token.lemma_.lower())
                    
                elif token.pos_=='ADJ':
                    adjs.append(token.lemma_.lower())

    # for printing out for inspection purposes
    if print_word_counts:
        for word_lst in [verbs, dobjs, nouns, adjs]:
            counter=collections.Counter(word_lst)
            print(counter)
    
    # take most common words of each form
    if len(verbs) > 0:
        verb = most_common(verbs, 1)[0][0]
    
    if len(dobjs) > 0:
        dobj = most_common(dobjs, 1)[0][0]
    
    if len(nouns) > 0:
        noun1 = most_common(nouns, 1)[0][0]
    
    if len(set(nouns)) > 1:
        noun2 = most_common(nouns, 2)[1][0]
    
    # concatenate the most common verb-dobj-noun1-noun2 (if they exist)
    label_words = [verb, dobj]
    
    for word in [noun1, noun2]:
        if word not in label_words:
            label_words.append(word)
    
    if '' in label_words:
        label_words.remove('')
    
    label = '_'.join(label_words)
    
    return label

In [124]:
def apply_and_summarize_labels(df, category_col):
    """
    Assign groups to original documents and provide group counts

    Arguments:
        df: pandas dataframe of original documents of interest to
            cluster
        category_col: str, column name corresponding to categories or clusters

    Returns:
        summary_df: pandas dataframe with model cluster assignment, number
                    of documents in each cluster and derived labels
    """
    
    numerical_labels = df[category_col].unique()
    
    # create dictionary of the numerical category to the generated label
    label_dict = {}
    for label in numerical_labels:
        current_category = list(get_group(df, category_col, label)['thought'])
        label_dict[label] = extract_labels(current_category)
        
    # create summary dataframe of numerical labels and counts
    summary_df = (df.groupby(category_col)['thought'].count()
                    .reset_index()
                    .rename(columns={'thought':'count'})
                    .sort_values('count', ascending=False))
    
    # apply generated labels
    summary_df['label'] = summary_df.apply(lambda x: label_dict[x[category_col]], axis = 1)
    
    return summary_df

In [110]:
def combine_ground_truth(df_clusters, df_ground, key):
    """
    Combines dataframes of documents with extracted and ground truth labels
    
    Arguments:
        df_clusters: pandas dataframe, each row as a document with corresponding extracted label
        df_ground: pandas dataframe, each row as a document with corresponding ground truth label
        key: str, key to merge tables on
        
    Returns:
        df_combined: pandas dataframe, each row as a document with extracted and ground truth labels
    """
    df_combined = pd.merge(df_clusters, df_ground, on=key, how = 'left')
    return df_combined

### need to check this function

In [111]:
def get_top_category(df_label, df_summary):
    """
    Returns a dataframe comparing a single model's results to ground truth
    label to evalute cluster compositions and derived label relative to labels
    and counts of most commmon ground truth category

    Arguments:
        df_label: pandas dataframe, each row as a document with extracted and ground truth labels
                  (result of `combine_ground_truth` function)
        df_summary: pandas dataframe with model cluster assignment, number
                    of documents in each cluster and derived labels
                    (result from `apply_and_summarize_labels` function)

    Returns:
        df_result: pandas dataframe with each row containing information on
                   each cluster identified by this model, including count,
                   extracted label, most represented ground truth label name,
                   count and percentage of that group
    """
    df_label_ground = (df_label.groupby('label')
                      .agg(top_ground_category=('category', lambda x:x.value_counts().index[0]), 
                           top_cat_count = ('category', lambda x:x.value_counts()[0]))
                      .reset_index())
    
    df_result = pd.merge(df_summary, df_label_ground, on='label', how='left')
    df_result['perc_top_cat'] = df_result.apply(lambda x: int(round(100*x['top_cat_count']/x['count'])), axis=1)
    
    return df_result


In [121]:

import collections
example_category = list(get_group(data_clustered_k_mean, 'k-mean', 0)['thought'])
extract_labels(example_category, True)


Counter({'forgot': 3, 'going': 2, 'messed': 2, 'gotten': 2, 'look': 2, 'pointed': 2, 'explaining': 1, 'overact': 1, 'reminded': 1, 'screwed': 1, 'explain': 1, 'shows': 1, 'bet': 1, 'spent': 1, 'return': 1, 'terrible': 1, 'rains': 1, 'learn': 1, 'behaved': 1, 'posted': 1, 'failed': 1, 'mixed': 1, 'got': 1, 'spilled': 1, 'ruined': 1, 'ended': 1, 'worried': 1})
Counter({'mistake': 4, 'job': 2, 'tire': 2, 'time': 2, 'message': 2, 'error': 2, 'material': 1, 'right': 1, 'thing': 1, 'review': 1, 'day': 1, 'audits': 1, 'piece': 1, 'calendar': 1, 'address': 1, 'experiment': 1, 'drink': 1, 'project': 1, 'problem': 1, 'business': 1, 'car': 1})
Counter({'work': 10, 'boss': 6, 'time': 3, 'tire': 3, 'failure': 3, 'month': 2, 'idiot': 2, 'color': 2, 'people': 2, 'artwork': 2, 'customer': 2, 'party': 2, 'jerk': 1, 'matter': 1, 'artist': 1, 'day': 1, 'mistake': 1, 'repairman': 1, 'curt': 1, 'gas': 1, 'station': 1, 'event': 1, 'date': 1, 'package': 1, 'place': 1, 'reagent': 1, 'car': 1, 'crash': 1, 'dau

'forgot_mistake_work_boss'

In [125]:
cluster_summary = apply_and_summarize_labels(data_clustered_k_mean, 'k-mean')
cluster_summary.head(20)


,k-mean,count,label
5,5,75,got_boyfriend_friend_husband
3,3,70,like_friend_girlfriend_partner
13,13,66,feel_thought_racing_time
1,1,65,got_grade_writer
7,7,60,feel_solo_failure_time
11,11,60,lose_job_failure
4,4,57,hate_friend_parent
2,2,52,hate_appearance_self_life
8,8,52,going_friend_person_time
10,10,49,going_sister_family


In [128]:
labeled_clusters = pd.merge(data_clustered_k_mean, cluster_summary[['k-mean', 'label']], on='k-mean', how = 'left')
labeled_clusters.head(19)


,thought,k-mean,label
0,Someone I trusted stole something valuable of ...,15,felt_party_friend_work
1,She doesn't respect me.,3,like_friend_girlfriend_partner
2,My friend is ignoring his recently-deceased wife.,5,got_boyfriend_friend_husband
3,he took me for granted!,5,got_boyfriend_friend_husband
4,We will not be able to complete the work that ...,16,felt_job_meeting_work
5,I failed our company & myself,11,lose_job_failure
6,Am I not explaining the material well?,0,forgot_mistake_work_boss
7,We won't be able to find the store before it c...,11,lose_job_failure
8,He doesn't love me.,5,got_boyfriend_friend_husband
9,Suicidal thoughts,13,feel_thought_racing_time


In [133]:
data_ground = results_k_mean[['thought', 'original_label']]

In [135]:
labeled_clusters = combine_ground_truth(labeled_clusters, data_ground, 'thought')
labeled_clusters.sample(10)


,thought,k-mean,label,original_label_x,original_label_y
93,I feel like a bad wife and mom,14,feel_time_mother_kid,"emotional reasoning,labeling","emotional reasoning,labeling"
530,"If I did not have friends, I would be lonely",8,going_friend_person_time,All-or-nothing thinking,All-or-nothing thinking
666,I run a business with my close friend and we w...,15,felt_party_friend_work,Mind reading,Mind reading
457,Going to college is not a big deal; anyone can...,1,got_grade_writer,Disqualifying the positive,Disqualifying the positive
493,People my age are more successful than I am,1,got_grade_writer,Comparing,Comparing
405,My niece's birthday party had twice the amount...,15,felt_party_friend_work,Comparing,Comparing
288,I'm late for the meeting. This shows what a je...,16,felt_job_meeting_work,Labeling,Labeling
454,"I ate more than I planned, so I blew my diet c...",17,gained_weight_diet_week,All-or-nothing thinking,All-or-nothing thinking
151,He's insulting me.,5,got_boyfriend_friend_husband,"mind reading,labeling,negative feeling or emot...","mind reading,labeling,negative feeling or emot..."
517,I must be really stupid to have these distorte...,13,feel_thought_racing_time,Personalization,Personalization


In [136]:
labeled_clusters[labeled_clusters['k-mean']==0]


,thought,k-mean,label,original_label_x,original_label_y
6,Am I not explaining the material well?,0,forgot_mistake_work_boss,"all-or-nothing thinking,personalizing","all-or-nothing thinking,personalizing"
46,I should have been perfect,0,forgot_mistake_work_boss,should statements,should statements
62,I didn't do as good a job as I thought,0,forgot_mistake_work_boss,disqualifying the positive,disqualifying the positive
69,I forgot something important. Again. My boss i...,0,forgot_mistake_work_boss,"catastrophizing,fortune telling","catastrophizing,fortune telling"
76,Why did I overact like this,0,forgot_mistake_work_boss,catastrophizing,catastrophizing
78,I messed up big time.,0,forgot_mistake_work_boss,catastrophizing,catastrophizing
80,they should have reminded me before I forgot,0,forgot_mistake_work_boss,should statements,should statements
85,I could have done so much better,0,forgot_mistake_work_boss,"personalizing,all-or-nothing thinking","personalizing,all-or-nothing thinking"
108,I'm mad at myself for being careless,0,forgot_mistake_work_boss,"emotional reasoning,labeling,negative feeling ...","emotional reasoning,labeling,negative feeling ..."
111,I screwed up again at work,0,forgot_mistake_work_boss,"negative feeling or emotion,all-or-nothing thi...","negative feeling or emotion,all-or-nothing thi..."


In [138]:
get_top_category(labeled_clusters, cluster_summary)


KeyError: "Column(s) ['category'] do not exist"

# Result and analysis
https://github.com/dborrelli/chat-intents/blob/main/notebooks/05-label-intents.ipynb
https://bard.google.com/chat/bf7d121f09db9a00

In [142]:
import spacy

# Chargez les données
data = pd.DataFrame({
    "phrase": ["She doesn't respect me", "he took me for granted!", "I'm always the one who has to do everything", "I'm so stupid", "I'm never going to be successful"]
})

# Enlevez les étiquettes
data["phrase"] = data["phrase"].str.replace("_", " ")

# Analysez les phrases
doc = nlp(data["phrase"].tolist())

# Afficher les sentiments des phrases
for sent in doc.sents:
    print(sent.sentiment)

SyntaxError: unexpected EOF while parsing (1787068298.py, line 12)

# Generate label with LLM

In [252]:
## Original Labelling

label_list = []
# json_dataset
with open('/Users/sylvainestebe/Code/nlp_project/cognitive_distortion_project/data/thinking_traps.jsonl') as f:
    df = pd.read_json(f, lines=True)

for i in range(len(df)):

#  Remove arrow
    modified_prompt = df["completion"][i].replace("->", "")
    label_list.append(modified_prompt)

# 18 labels

label_list
negative_label = remove_same_sentences(label_list)
negative_label


[' Overgeneralization',
 ' Mind reading',
 ' Fortune telling',
 ' Labeling',
 ' Should statements',
 ' All-or-nothing thinking',
 ' Emotional reasoning',
 ' Catastrophizing',
 ' Personalization',
 ' Disqualifying the positive',
 ' Overgeneralizing',
 ' Comparing',
 ' Blaming',
 ' Not distorted',
 ' Negative feeling or emotion',
 ' Personalizing',
 ' Comparing and Despairing',
 ' Disqualifying the Positive']

## Generated new label
Don't forget to generate token with hugginface https://huggingface.co/settings/tokens
huggingface-cli login




In [253]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("epfl-llm/meditron-7b")
model = AutoModelForCausalLM.from_pretrained("epfl-llm/meditron-7b")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 8/8 [00:37<00:00,  4.71s/it]
